[View in Colaboratory](https://colab.research.google.com/github/sabumjung/Tensorflow/blob/master/JA_01_Simple%20Policy.ipynb)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
#List out our bandit arms. 
#Currently arm 4 (index #3) is set to most often provide a positive reward.
#해당 밴딧암의 값보다 크면 양의 reward를 받고, 작으면 음의 reward를 받는다.
bandit_arms = [0.2,-5,-0.2,-2]
num_arms = len(bandit_arms)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [0]:
#graph를 reset하고 정의한다.
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. 
#가중치 행렬 : 초기 1로 설정함 [1,1,1,1]
weights = tf.Variable(tf.ones([num_arms]))
#출력 : 가중치연산결과에 대해 softmax함수를 적용함
#graph를 완성하여 output에 넣는다.
#weights에 대한 softmax활성화함수를 계산하는 연산을 정의하고 graph인 output에 저장함
output = tf.nn.softmax(weights)

#The next six lines establish the training proceedure. 
#We feed the reward and chosen action into the network to compute the loss, and use it to update the network.
#모델링 훈련 파라미터(reward, action값을 저장하기 위한 플레이스홀더 정의)
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)

responsible_output = tf.slice(output,action_holder,[1])
loss = -(tf.log(responsible_output)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

In [4]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_arms) #Set scoreboard for bandit arms to 0.

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose action according to Boltzmann distribution.
        actions = sess.run(output)
       
        #actions구성요소에서 랜덤선택수행, 각 구성요소 선택확률은 p값임  
        a = np.random.choice(actions, p=actions)
        #조건문 actions == a의 결과에서 가장 큰값인 1(TRUE)을 갖는 열인덱스를 찾아내서 action에 저장함
        action = np.argmax(actions == a)
        # print(actions, actions==a, action)

        #몇번째 밴딧암을 선택할지 pullBandit의 인수로 전달함
        reward = pullBandit(bandit_arms[action]) #Get our reward from picking one of the bandit arms.
        
        #신경망을 업데이트함
        #reward, action이력을 저장함
        _,resp,ww = sess.run([update,responsible_output,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #score이력을 저장함
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
        i+=1
print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [1. 0. 0. 0.]
Running reward for the 4 arms of the bandit: [ 1. 10.  1. 15.]
Running reward for the 4 arms of the bandit: [ 6. 29.  4. 20.]
Running reward for the 4 arms of the bandit: [ 6. 41.  9. 31.]
Running reward for the 4 arms of the bandit: [ 3. 53.  5. 44.]
Running reward for the 4 arms of the bandit: [-5. 68.  4. 52.]
Running reward for the 4 arms of the bandit: [-1. 85. 10. 67.]
Running reward for the 4 arms of the bandit: [ 0. 96. 13. 80.]
Running reward for the 4 arms of the bandit: [ -4. 111.  20.  88.]
Running reward for the 4 arms of the bandit: [ -8. 126.  26. 101.]
Running reward for the 4 arms of the bandit: [-12. 139.  29. 117.]
Running reward for the 4 arms of the bandit: [-15. 155.  32. 133.]
Running reward for the 4 arms of the bandit: [-17. 170.  31. 147.]
Running reward for the 4 arms of the bandit: [-21. 192.  32. 156.]
Running reward for the 4 arms of the bandit: [-27. 202.  35. 171.]
Running reward for the 4 arms o